In [136]:
import json
import pandas as pd
from energyid import JSONClient
from datetime import datetime

example json
```json
{
  "EnergyID_CLIENTID": "ID",
  "EnergyID_CLIENTSECRET": "SECRET",
  "EnergyID_USERNAME": "USERNAME",
  "EnergyID_PASSWORD": "PASSWORD",
  "RECORDS": {
    "RECORD_1": "EA-NUMBER"
  }
}

with open("credentials.yaml", "r") as f:
    credentials = yaml.safe_load(f)

In [137]:
# Load credentials (clientID, clientSecret, username, password) from json file
with open("secrets.json") as f:
    credentials = json.load(f)

In [138]:
# Create Client and init with client id and client secret

client = JSONClient(
    client_id=credentials["client_id"], client_secret=credentials["client_secret"]
)

In [139]:
# Login with username (email) and password

client.authenticate(username=credentials["username"], password=credentials["password"])

In [ ]:
# Download a members info
# default member is ME

member = client.get_member()
member

In [ ]:
# Get all records for the member

records = member.get_records()
pd.DataFrame(records)

In [ ]:
# Select the first record

record = records[0]
pd.Series(record)

In [ ]:
# Get Energy Use data for the record

record.get_data(name="energyUse")